In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

os.environ['THEANO_FLAGS'] = "device=cuda0"

In [ ]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
from lproc import rmap, subset, chunk_load
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sltools.utils import pad_seq, dichotomy

from experiments.siamese_oneshot.a_data import durations, labels, transformations, \
    train_subset, val_subset
from experiments.siamese_oneshot.b_preprocess import feat_seqs
from experiments.siamese_oneshot.c_model import build_model

np.set_printoptions(linewidth=100)

# Load dataset

In [ ]:
max_time = 128
feat_seqs = rmap(lambda s: pad_seq(s, max_time), feat_seqs)

feat_seqs_train = subset(feat_seqs, train_subset)
transformations_train = subset(transformations, train_subset)
labels_train = labels[train_subset].astype(np.int32)
durations_train = durations[train_subset]

feat_seqs_val = subset(feat_seqs, val_subset)
transformations_val = subset(transformations, val_subset)
labels_val = labels[val_subset].astype(np.int32)
durations_val = durations[val_subset]

input_shape = feat_seqs[0][0].shape

def generate_pairs(transformations, labels, vocabulary, n, positive_ratio=0.6):
    # precompute where to look for positive pairs
    where_labels = {l: np.where(labels == l)[0] for l in vocabulary}
    where_not_labels = {l: np.where(labels != l)[0] for l in vocabulary}
    
    pairs = np.empty((n, 2), dtype=np.uint64)
    positive = np.empty((n,), dtype=np.bool)
    
    for k in range(n):
        i = k % len(labels)  # simply pick left items iteratively
        l = labels[i]
        p = np.random.random() < positive_ratio
        if p:
            j = np.random.choice(where_labels[l])
            while transformations[j][0] == transformations[i][0]:
                j = np.random.choice(where_labels[l])
        else:
            j = np.random.choice(where_not_labels[l])
        
        pairs[k] = i, j
        positive[k] = p
    
    return pairs, positive

# Build model

In [ ]:
batch_size = 16

model = build_model(feat_seqs[0][0].shape, batch_size, max_time)
l_linout = model['l_linout']
l_in_left, l_in_right = model['l_in']
l_duration_left, l_duration_right = model['l_duration']
linout = lasagne.layers.get_output(l_linout)

# Build training routines
targets = T.vector('targets')
l_rate_var = T.scalar('l_rate')
loss = T.switch(targets > .1,
                .5 * linout ** 2,
                .5 * T.maximum(0, 1 - linout) ** 2).sum()
params = lasagne.layers.get_all_params(l_linout, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=l_rate_var)
update_fn = theano.function([l_in_left.input_var, l_duration_left.input_var,
                             l_in_right.input_var, l_duration_right.input_var,
                             targets, l_rate_var],
                            outputs=loss, updates=updates)

linout2 = lasagne.layers.get_output(l_linout, deterministic=True)
loss2 = T.switch(targets > .1,
                 .5 * linout2 ** 2,
                 .5 * T.maximum(0, 1 - linout2) ** 2)
predict_fn = theano.function([l_in_left.input_var, l_duration_left.input_var,
                              l_in_right.input_var, l_duration_right.input_var,
                              targets],
                             outputs=[linout2, loss2])

running_loss = 0

buffers = [np.zeros((4 * batch_size, max_time) + input_shape, dtype=np.float32),
           np.zeros((4 * batch_size,), dtype=np.int32),
           np.zeros((4 * batch_size, max_time) + input_shape, dtype=np.float32),
           np.zeros((4 * batch_size,), dtype=np.int32),
           np.zeros((4 * batch_size,), dtype=np.bool)]

# Run training iterations

In [ ]:
l_rate = 1e-3

for e in range(20):
    pairs, tgts = generate_pairs(transformations_train, labels_train, np.unique(labels), 
                                 len(train_subset) * 2, .6)
    x1 = rmap(lambda pair: feat_seqs_train[pair[0]], pairs)
    x2 = rmap(lambda pair: durations_train[pair[0]], pairs)
    x3 = rmap(lambda pair: feat_seqs_train[pair[1]], pairs)
    x4 = rmap(lambda pair: durations_train[pair[1]], pairs)
    for i, (xl, dl, xr, dr, tgt) in enumerate(chunk_load([x1, x2, x3, x4, tgts], buffers, 
                                                          chunk_size=batch_size, pad_last=False)):
        if len(tgt) != batch_size:
            continue
        batch_loss = update_fn(xl, dl, xr, dr, tgt, l_rate)
        running_loss = .98 * running_loss + .02 * batch_loss
        if i % 30 == 0:
            print("\rloss: {}".format(running_loss), end="", flush=True)

    print("\repoch {:3d} loss: {}".format(e, running_loss))

# Preview results

In [ ]:
def preview_perfs(feat_seqs_, durations_, pairs_, tgts_, predict_fn_, buffers_, thres_=None):
    x1 = rmap(lambda pair: feat_seqs_[pair[0]], pairs)
    x2 = rmap(lambda pair: durations_[pair[0]], pairs)
    x3 = rmap(lambda pair: feat_seqs_[pair[1]], pairs)
    x4 = rmap(lambda pair: durations_[pair[1]], pairs)
    all_preds = np.empty((len(tgts_) - len(tgts_) % batch_size,))
    all_losses = np.empty((len(tgts_) - len(tgts_) % batch_size,))
    all_targets = np.empty((len(tgts_) - len(tgts_) % batch_size,))
    i = 0
    for xl, dl, xr, dr, tgt in chunk_load([x1, x2, x3, x4, tgts], buffers_, 
                                           chunk_size=batch_size, drop_last=True):
        all_preds[i:i + batch_size], all_losses[i:i + batch_size] = \
            predict_fn_(xl, dl, xr, dr, tgt)
        all_targets[i:i + batch_size] = tgt
        i += len(tgt)
    
    if thres_ is None:
        thres_ = dichotomy(
            lambda t: 1 - np.mean(all_preds[all_targets > .5] > t)
                      / np.mean(all_preds[all_targets < .5] < t),
            min(all_preds), max(all_preds), it=20)

    print("fnr = ", np.mean(all_preds[all_targets < .5] < thres_))
    print("fpr = ", np.mean(all_preds[all_targets > .5] > thres_))
    print("thres = ", thres_)

    # observe training results
    plt.figure(figsize=(12, 3))
    
    plt.subplot(1, 2, 1)
    bins = np.linspace(0, 1, 40)
    pos, _ = np.histogram(all_preds[all_targets == 1], bins=bins)
    neg, _ = np.histogram(all_preds[all_targets == 0], bins=bins)
    plt.bar(bins[:-1], pos / pos.sum(), width=.025, color='red', alpha=.5)
    plt.bar(bins[:-1], neg / neg.sum(), width=.025, color='blue', alpha=.5)
    plt.plot([thres_, thres_], [0, 1])
    
    plt.subplot(1, 2, 2)
    fpr, tpr, _ = roc_curve(all_targets > .5, all_preds)
    plt.plot(tpr, fpr)
    plt.gca().set_aspect('equal')
    
    plt.show()
    
    return all_targets, all_preds, thres_


pairs, tgts = generate_pairs(transformations_train, labels_train, np.unique(labels), 
                             len(train_subset) * 2, .6)
all_targets, all_preds, thres = preview_perfs(
    feat_seqs_train, durations_train, pairs, tgts, predict_fn, buffers, None)

pairs, tgts = generate_pairs(transformations_val, labels_val, np.unique(labels), 
                             len(val_subset) * 2, .6)
all_targets, all_preds, thres = preview_perfs(
    feat_seqs_val, durations_val, pairs, tgts, predict_fn, buffers, thres)

# Preview training data

In [ ]:
from sklearn.metrics import confusion_matrix

pairs, tgts = generate_pairs(transformations_train, labels_train, np.unique(labels),
                             len(train_subset) * 2, positive_ratio)
cnf = confusion_matrix(labels_train[pairs[:, 0]], labels_train[pairs[:, 1]])
print(cnf)